# DapticsClient - Introduction

This notebook contains an interactive introduction to the Python DapticsClient class,
a simplified interface for accessing the Daptics GraphQL API for the optimization of
experimental design.

Documentation for using the DapticsClient class (implemented in the daptics_client.py
file in this folder) is included as comment lines in the interactive Python cells of
this notebook.

For additional help or information, please visit or contact Daptics.

On the web at https://daptics.ai
By email at support@daptics.ai

Daptics API Version 0.7.2  
Copyright (c) 2019 Daptics Inc.

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), the rights to use, copy, modify, merge, publish, and/or distribute, copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

You do not have the right to sub-license or sell copies of the Software.

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
# Step 1. Create a DapticsClient object and connect to the API server.

# Before running this project, please make sure that your Jupyter
# Python environment has these required packages installed:
#
#   chardet
#   urllib3
#   requests
#   gql
#
# You will also need a validated user account on the Daptics API server.
# You can create an account by clicking the `Get Started for Free` button at
# https://daptics.ai
#
# See the 01_README.ipynb notebook in this folder for more information.

# Import the daptics_client module
# Requirements are Python 3, and the `gql` and `requests` libraries
from daptics_client import DapticsClient

# Create a client object and try to connect to the daptics API server.
# The client constructor takes a URL argument that specifies the scheme,
# host and port number of the API server.

# Note: do not specify the path on the server; the DapticsClient object
# will take care of that for us.
api_host = 'http://inertia.protolife.com:8080'
daptics = DapticsClient(api_host)

# The `connect` method will attempt to connect to the /api path on the
# API server and obtain the GraphQL schema.
daptics.connect()

# Show the result. After connecting, the daptics object should have a `gql` attribute.
# Let's look at the data stored in the `gql` attribute by printing its `__dict__`.
# We can see that the gql library has introspected all the GraphQL type,
# query and mutation information exposed by the API.
print(daptics.gql.__dict__)

In [ ]:
# Step 2. Log in to the API server to obtain an access token.

# The `login` method takes two string arguments, the user's
# `email` and `password`.

# Note: Use the real email address and password you used when you created
# your account on the daptics.ai website.

email = 'YOUR_EMAIL@YOUR_DOMAIN.com'
password = 'YOUR_PASSWORD'
data = daptics.login(email, password)

# Show the result. The `daptics` object will remember the access token
# and use it for all subsequent API requests. The `daptics` object
# will also remember the `userId` that corresponds to the email
# address.
print(data)

In [ ]:
# Step 3. Show the names and ids of all the sessions belonging to the
# user.

data = daptics.list_sessions()
print(data)

# Look for the last session in the result.
sessions = data['sessions']
last_session = [s for s in sessions if s['active']][-1]
print('Found {} sessions belonging to user'.format(len(sessions)))
print('Last active session is {}'.format(last_session))

In [ ]:
# Step 4. Reconnect to a daptics session on the server.

# The `reconnect_session` method takes one argument:
#    `session_id` - the ID string of a previously created session
#
# We will use the last session found in the previous step. If you prefer,
# you may copy and paste a quoted sessionId value from the previous step.

session_id = last_session['sessionId']
data = daptics.reconnect_session(session_id)

# Show the result. The `daptics` object will remember the `sessionId`.
print(data)

In [ ]:
# Step 5. Fetch the experiments for a previous generation.

# Show the current generation number for the session, saved in the `daptics` object.
print('Current generation: {}'.format(daptics.gen))

# The `get_experiments` method takes an optional argument:
#    `gen` - the generation number for the experiments to be fetched. Use zero to
# fetch any initial experiments. If gen is not supplied (or is None), the design for the
# current generation will be retrieved.

# Get a previous generation.
gen = 2
data = daptics.get_experiments(gen)

# Show the result.
print(data)

In [ ]:
# Step 6. Fetch analytics for the latest generation.

# Show the current generation number for the session, saved in the `daptics` object.
print('Current generation: {}'.format(daptics.gen))

# The `get_analytics` method takes no arguments, and returns a list of URLs.
data = daptics.get_analytics()

# Show the result.
print(data)

# Save the result in a variable for use in the next step.
files = data['createAnalytics']['files']

In [ ]:
# Step 7. Download one of the analytics files.

# The `save_analytics_file` method takes two arguments:
#    `url` - The URL returned by the `get_analytics` method.
#    `save_as` - A file path to write the file to.
#
# We will extract the URL and filename from the `files` variable
# we set in the previous step.
file = files[0]
url = file['url']
fname = file['filename']
data = daptics.save_analytics_file(url, save_as=fname)

# Show the result, a response object.
print(data)

# Show the length of the file downloaded.
import os
size = os.path.getsize(fname)
print('Size of {0} is {1} bytes'.format(fname, size))

In [ ]:
# Step 8. Display the PDF in an iframe.

# We'll use a helper class that exposes a `_repr_html_` method.
# We use "embed" tag because Chrome will not let us show an iframe.
class PDF(object):
  def __init__(self, path, height=400):
    self.path = path
    self.height = height

  def _repr_html_(self):
    return '<embed type="application/pdf" src="{0}" width="100%" />'.format(self.path)

  def _repr_latex_(self):
    return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.path)

# Initialize a PDF object, and let Jupyter notebook show us the iframe representation.
PDF(fname)